# Setting Jubilee Tool Parking Positions
To pick up a tool, we need to define the X and Y position where the tool is parked to be able to pick it up and drop it off. This notebook walks through the steps outlined [here](https://www.jubilee3d.com/index.php?title=Setting_Tool_Parking_Positions) to set tool parking positions!



### 0. Notebook Setup
We'll need to send commands to the machine–the following cells set up this communication.

In [ ]:
# Import required modules
from science_jubilee.Machine import Machine
from science_jubilee.tools.Tool import Tool
from CalibrationControlPanel import make_control_panel
import os
import sys
from jinja2 import Environment, FileSystemLoader, select_autoescape

In [ ]:
# Establish connection with machine
m = Machine(address="192.168.1.2") # edit the address as necessary

In [ ]:
# Change the values in this cell!
tool_number = 0           # What is the number of the tool you are setting up? 
tool_name = "Pipette"     # Set a human-readable name for this tool

# Pickup and park moves should be 'manhattan' style to avoid collisions
# That is, we should move in a square block (X & Y seperately) rather than along the shortest path
# This ensures we don't hit parking posts by moving directly
# A default value of 60 works here!
manhattan_offset = 60 

control_panel = make_control_panel(m) # This will help us make fine-grain alignment adjustments

### 1. Tool Definition
We need to give the Duet some information about the tool we are going to set up. First, we need to define the tool we're setting up by providing an index and a name. We can do this by adding an `M563` command to the `config.g` file. 

Run the following cell and copy the output. Paste it in your `config.g file` (to do so, go the the DuetWebControl interface, select System in the left hand toolabr, and then click on the `config.g` file). There will be a section almost at the bottom titled 'Tool Definitions'; paste the `M563` command there.

After saving the file, select 'restart the mainboard' on the pop-up window that appears.

In [ ]:
print(f'M563 P{tool_number} S"{tool_name}"')

You'll need to rehome the machine after restarting the mainboard. Run the following cell to do so! Make sure your bed plate is clear.

In [ ]:
m.home_all()

### 2. Coursely place your parking post.
Take your parking post assembly and loosely fasten it to the rail approximately where you want it; you should still be able to move the parking post around to adjust it for X-axis position and parking post height. You'll need to align 2 tee-nuts on the Y axis rail, and you can access the m5 screws through the holes on top of the parking post.

### 3. Move the build plate
Drop your build plate down (positive Z values) such that any tools you’re about to mount will fully clear the bed.

In [ ]:
m.move_to(z=150)

### 4. Jog the carriage to somewhere close to the center of the printer bed.

In [ ]:
m.move_to(x=150, y=150)

### 5. With no active tool, manually place your tool against the lock carriage, and lock the tool onto the carriage
Run the following tool lock cell once you have placed the tool against the lock carriage.

In [ ]:
m.tool_lock()

### 6. Jog the carriage to roughly where you want to end up across the X-axis.
Move to the approximate position where your parking post is.

In [ ]:
m.move_to(x=100) # change this value to approximately line up with the parking post

### 6. Jog the carriage forward in the Y-axis until its roughly 10mm away parking post assembly. Make sure the tool is totally clear of the dowel pins for now.
Move to the approximate position, then use the control panel to line things up.

In [ ]:
m.move_to(y=200) # move closer to the parking post

In [ ]:
# Now use the control panel to line things up in X and Y
control_panel

The tool should now be just in front of the parking post. We'll save this value for later!

In [ ]:
# We'll save this y position as the 'y clear' value
# Run this cell to save the position; we'll use it later!
y_clear = float(m.get_position()['Y'])
y_clear

### 8. Line up the parking post on the rail to the tool across the X-axis.
You can do this by physically moving the parking post along the rail

### 9. Slowly jog the carriage forward in the Y-axis until the tool starts engaging the parking dowel pins.
You can do this by incrementing in y using the control panel.

In [ ]:
control_panel

### 10. Take a look to make sure everything is lined up: dowel pins, parking wings, etc.
You might have to adjust the height of the parking post to get things lined up correctly. To do this, make sure the two screws on the back of the parking post are loosened, and access the height adjustment screw through the hole in the top of the parking post. Tightening will bring the post down, and loosening will raise it.

### 11. Jog the carriage further in the Y-axis in small (1mm) increments until you’ve achieved a full dock. Depending on your tool, your tool wings may or may not make contact with the pack of the parking post.

In [ ]:
control_panel

### 12. Adjust the position in the X-axis to your liking (not by hand, we don’t want to lose position).
You can move +/-0.1mm in x to line things up.

In [ ]:
control_panel

### 13. Once your tool and parking post are fully engaged and in the final position you want them to be, we'll save the X and Y values reported for the printer’s current position.

In [ ]:
# We'll save this position to use in a couple files
pos = m.get_position()
x_park = float(pos['X'])
y_park = float(pos['Y'])
x_park, y_park

### 14. Tighten down the tee-nuts to lock in your X-axis position for this tool.

### 15. Adjust your parking post height adjustment screw until you’re satisfied that the tool is properly supported and parallel to the Y-axis rails of your printer. Tighten down the screws for the height adjustment on the back of the parking post to lock it in.

### 16. Unlock the tool from the carriage using the following cell

In [ ]:
m.tool_unlock()

### 17. Jog the now-decoupled carriage backwards in the Y-axis direction until it fully clears your tool(s) and parking post(s).

In [ ]:
# Move back in until the tool is comfortably cleared. 
control_panel

### 18. Update the relevant files for the tool you have just set up.
There are 3 files which need to be updated with the values you've found. The files (or 'macros') are run each time we pick up a tool. We'll use template files to make this quicker–run the following cell to set up the templating engine. 

In [ ]:
env = Environment(loader= FileSystemLoader("templates"))

The `tpostx.g` file is called for tool number 'x' after the firmware thinks tool number x is selected. From the duet console, navigate to system-->tpostx.g, where x is the number of the tool you are currently calibrating--create the file if it doesn't yet exist. Run the following cell and replace the contents (i.e. copy-paste) with the output produced:

In [ ]:
template = env.get_template("tpost.g")
tpost_content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(tpost_content)

Similarly, `tfreex.g` runs at the start of a toolchange if the current tool is tool-x. Again, run the following cell and copy-paste the output into tfreex.g, replacing any contents that were there or creating the file if it doesn't exist:

In [ ]:
template = env.get_template("tfree.g")
tfree_content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(tfree_content)

Finally, the tprex.g file is where the carriage will move to immediately before & after picking up a tool. Run the following call and replace tpre.x with the output or create it if it doesn't exist:

In [ ]:
template = env.get_template("tpre.g")
tpre_content = template.render(tool_number=tool_number, x_park=x_park, 
                          y_park=y_park, y_clear=y_clear, manhattan_offset = manhattan_offset)

print(tpre_content)

### 19. Try out a tool change to the new parking post using the Tx command!
Test both selecting a tool and parking it. While testing, check to see if your setup needs a few tweaks to make sure the lock pin fully engages the ramp (change the Y-axis value used in tpost - higher Y values move the locking carriage closer to the tool dock) upon locking. Check to see if the tool has been docked correctly after unlocking (change the Y-axis values in tfree to adjust this to your needs).

In [ ]:
# Load your tool
tool = Tool(tool_number, tool_name)
m.load_tool(tool)

In [ ]:
m.pickup_tool(tool)

In [ ]:
m.park_tool()